In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# Ask user for any Jumia category or search URL
base_url = input("Paste any Jumia category or search URL: ").strip()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36'
}
# this headers = {'User-Agent':''} this will make the jumia site think the scrap request is comming from a normal user using a browser and not python or an AI

all_products = []

# Loop through multiple pages until no products found
for page in range(1, 51):  # limit to 50 pages max
    if "?" in base_url:
        url = f"{base_url}&page={page}"
    else:
        url = f"{base_url}?page={page}"

# so this says loop through the pages of the base_url, max 51 pages but as you loop through it,
# check if there is a ? query parameter, if there is, add page to is and loop through all pages
# up to 51, but if there is no query parameter ? so we start the query parameter with the page 
    
    
    print(f"Scraping {url}")
    res = requests.get(url, headers=headers)
    
    if res.status_code != 200:
        print(f"Failed to load page {page}")
        break
    
    soup = BeautifulSoup(res.text, 'lxml')
    products = soup.find_all('article', class_='prd')
    
    if not products:  # stop if no more products
        break
    
    for product in products:
        name_tag = product.find('h3', class_='name')
        price_tag = product.find('div', class_='prc')
        rating_tag = product.find('div', class_='stars _s')
        
        name = name_tag.text.strip() if name_tag else None
        price_text = price_tag.text.strip().replace('₦', '').replace(',', '') if price_tag else None
        price = int(price_text) if price_text and price_text.isdigit() else None
        rating = rating_tag.text.strip() if rating_tag else None
        
        all_products.append({
            "Product Name": name,
            "Price (NGN)": price,
            "Rating": rating
        })
    
    time.sleep(1)  # polite delay

# Create DataFrame
df = pd.DataFrame(all_products)

# Save to CSV
df.to_csv("jumia_products.csv", index=False)

print("\nScraping completed!")
print(df.head())